In [ ]:
def tochar(num):
    "Converts a number in the range 0..25 into a letter of the alphabet"
    return chr(num + 65) # 65 is the ASCII code for the letter A

def tonum(char):
    "Converts a letter of the alphabet into a number in the range 0..25"
    return ord(char) - 65 # 65 is the ASCII code for the letter A
    
def clean_string(string):
    return "".join([c for c in string.upper() if "A" <= c <= "Z"])

def encrypt(key, letter):
    "Encrypts a letter using a key"
    num_letter = tonum(clean_string(letter))
    num_letter = (num_letter + key)%26
    char_letter = tochar(num_letter)
    return char_letter

def decrypt(key, letter):
    "Decrypts a letter using a key"
    num1_letter = tonum(clean_string(letter))
    num1_letter = (num1_letter-key)%26
    char1_letter = tochar(num1_letter)
    return char1_letter

def caesar_decrypt(key, str):
    "Decrypts a Caesar Cipher with a given ciphertext and key"
    for i in str:
        print(decrypt(key, i), end='')

def vigenere(key, message):
    "Decrypts a Vigenere Cipher with a given ciphertext and key"
    key_length = len(key)
    key_num = [tonum(i) for i in key]
    message_num = [tonum(i) for i in message]
    plaintext = ''
    for i in range(len(message)):
        value = (message_num[i] - key_num[i % key_length])%26
        plaintext += tochar(value)
    return plaintext


def gcd(a,b):
    "Finds the Greatest Common Divisor of two numbers
    remainder = 1
    while  remainder != 0:
        quotient = divmod(a,b)[0]
        remainder = divmod(a,b)[1]
        a = b
        b = remainder
    return a

def euclidean(a, b):
    "Returns g, x, y, such that g = gcd(a, b) and ax + by = g"
    if a == b == 0:
        raise ValueError("GCD of 0 and 0 is undefined")
    x1, x2 = 1, 0
    y1, y2 = 0, 1
    while True:
        q, r = divmod(a, b)
        if r == 0:
            break
        x1, x2 = x2, x1 - x2*q
        y1, y2 = y2, y1 - y2*q
        a, b = b, r
    if b < 0:
        return -b, -x2, -y2
    return b, x2, y2

def extended_euclid(a,b):
    if a == 0:
        return b, 0, 1

    gcd, u, v = extended_euclid(b%a, a)
    x = v - (b//a)*u
    y = u
    return gcd, x, y


def inverse(a, n):
    "Returns the inverse of a modulo n, raises ValueError if not coprime"
    d, x, y = euclidean(a, n)
    if d == 1:
        return x % n
    raise ValueError(f"{a} is not invertible modulo {n}")

def crt(residues, moduli):
    """Performs the Chinese Remainder Theorem
    Returns a pair (x, n) where n is the product of the moduli in the second
    argument, and x is congruent to each residue in the first argument modulo
    the corresponding modulus from the second argument.
    Note that if you have the residues and moduli already paired off, you can
    just "un-pair" them with zip, e.g.:
    crt(*zip((2, 5), (4, 7), (1, 11))) # returns (67, 385)
    This will raise ValueError if the moduli are not pairwise relatively prime.
    """
    bigresidue = 0
    bigmodulus = functools.reduce(operator.mul, moduli, 1)
    for a, m in zip(residues, moduli):
        y = bigmodulus // m
        z = inverse(y, m) # This will raise ValueError if y is not coprime to m
        bigresidue = (bigresidue + a*y*z) % bigmodulus
    return bigresidue, bigmodulus

def RSA(n,p,q,e,c):
    "Decrypts a piece of ciphertext encrypted with RSA"
    phi_n = (p-1)*(q-1)
    gcd, phi_neg, e_neg = extended_euclid(phi_n,e) 
    d = e_neg
    m = pow(c,d,n)
    return m